<a href="https://colab.research.google.com/github/indrajeetapache/Innovation-Hub/blob/main/Columnlinage/Column_linage_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
1. We will try to mimic the column lineage with LLM.
2. Spark will be used in this session and to provide the lineage etween source and target we will LLM
***




In [1]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, sum as spark_sum, avg, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql import functions as F
from io import StringIO
from pyspark.sql import DataFrame
import sys

In [2]:
# Initialize Spark
spark = SparkSession.builder \
    .appName("DAG_Lineage_Extraction") \
    .config("spark.sql.adaptive.enabled", "false") \
    .getOrCreate()

# Create sample data
sample_data = [
    ("John", "Doe", 25, 50000.0, "Engineering"),
    ("Jane", "Smith", 30, 75000.0, "Marketing"),
    ("Bob", "Johnson", 35, 60000.0, "Engineering"),
    ("Alice", "Brown", 28, 80000.0, "Sales"),
    ("Charlie", "Wilson", 32, 70000.0, "Marketing")
]

schema = StructType([
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", DoubleType(), True),
    StructField("department", StringType(), True)
])

# Create initial DataFrame
df_original = spark.createDataFrame(sample_data, schema)

In [3]:
df_original.show(10,0)

+----------+---------+---+-------+-----------+
|first_name|last_name|age|salary |department |
+----------+---------+---+-------+-----------+
|John      |Doe      |25 |50000.0|Engineering|
|Jane      |Smith    |30 |75000.0|Marketing  |
|Bob       |Johnson  |35 |60000.0|Engineering|
|Alice     |Brown    |28 |80000.0|Sales      |
|Charlie   |Wilson   |32 |70000.0|Marketing  |
+----------+---------+---+-------+-----------+



In [4]:
print("Original DataFrame:")
df_original.show()

# Transformation 1: Create derived columns
df_transformed = df_original \
    .withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name"))) \
    .withColumn("salary_category",
                F.when(col("salary") > 70000, "High")
                .when(col("salary") > 50000, "Medium")
                .otherwise("Low")) \
    .withColumn("age_group",
                F.when(col("age") < 30, "Young")
                .otherwise("Senior"))
print("\nTransformed DataFrame:")
df_transformed.show()

Original DataFrame:
+----------+---------+---+-------+-----------+
|first_name|last_name|age| salary| department|
+----------+---------+---+-------+-----------+
|      John|      Doe| 25|50000.0|Engineering|
|      Jane|    Smith| 30|75000.0|  Marketing|
|       Bob|  Johnson| 35|60000.0|Engineering|
|     Alice|    Brown| 28|80000.0|      Sales|
|   Charlie|   Wilson| 32|70000.0|  Marketing|
+----------+---------+---+-------+-----------+


Transformed DataFrame:
+----------+---------+---+-------+-----------+--------------+---------------+---------+
|first_name|last_name|age| salary| department|     full_name|salary_category|age_group|
+----------+---------+---+-------+-----------+--------------+---------------+---------+
|      John|      Doe| 25|50000.0|Engineering|      John Doe|            Low|    Young|
|      Jane|    Smith| 30|75000.0|  Marketing|    Jane Smith|           High|   Senior|
|       Bob|  Johnson| 35|60000.0|Engineering|   Bob Johnson|         Medium|   Senior|
|   

In [5]:
df_aggregated = df_transformed \
    .groupBy("department", "salary_category") \
    .agg(
        spark_sum("salary").alias("total_salary"),
        avg("age").alias("avg_age"),
        count("*").alias("employee_count")
    )

print("\nAggregated DataFrame:")
df_aggregated.show()


Aggregated DataFrame:
+-----------+---------------+------------+-------+--------------+
| department|salary_category|total_salary|avg_age|employee_count|
+-----------+---------------+------------+-------+--------------+
|Engineering|            Low|     50000.0|   25.0|             1|
|      Sales|           High|     80000.0|   28.0|             1|
|  Marketing|         Medium|     70000.0|   32.0|             1|
|Engineering|         Medium|     60000.0|   35.0|             1|
|  Marketing|           High|     75000.0|   30.0|             1|
+-----------+---------------+------------+-------+--------------+



**Function to extract dag information**

In [6]:
# Function to extract DAG information
def extract_dag_info(dataframe, stage_name):
    """Extract both logical and physical plan information"""

    # Method 1: Using explain() to capture plans
    from io import StringIO
    import sys

    # Capture explain output
    old_stdout = sys.stdout
    sys.stdout = buffer = StringIO()

    # Get extended plan information
    dataframe.explain(extended=True)
    plan_output = buffer.getvalue()

    sys.stdout = old_stdout

    # Split the explain output into logical and physical parts
    lines = plan_output.split('\n')
    logical_start = -1
    physical_start = -1

    for i, line in enumerate(lines):
        if "Parsed Logical Plan" in line or "Analyzed Logical Plan" in line:
            logical_start = i
        elif "Physical Plan" in line:
            physical_start = i
            break

    logical_plan_str = '\n'.join(lines[logical_start:physical_start]) if logical_start != -1 else "Not found"
    physical_plan_str = '\n'.join(lines[physical_start:]) if physical_start != -1 else "Not found"

    dag_info = {
        "stage_name": stage_name,
        "logical_plan": logical_plan_str,
        "physical_plan": physical_plan_str,
        "full_explain": plan_output,
        "schema": [{"name": field.name, "type": str(field.dataType)} for field in dataframe.schema.fields]
    }

    return dag_info


**TEXT for LLM**

In [7]:
# Function to extract transformation details for LLM
def extract_transformation_context(df_before, df_after, transformation_type):
    """Extract context that LLM can use for lineage analysis"""

    before_columns = [field.name for field in df_before.schema.fields]
    after_columns = [field.name for field in df_after.schema.fields]

    transformation_context = {
        "transformation_type": transformation_type,
        "input_columns": before_columns,
        "output_columns": after_columns,
        "new_columns": list(set(after_columns) - set(before_columns)),
        "dropped_columns": list(set(before_columns) - set(after_columns))
    }

    return transformation_context

**Extracted DAG information**

In [8]:
print("\n" + "="*50)
print("EXTRACTING DAG INFORMATION")
print("="*50)

original_dag = extract_dag_info(df_original, "original")
transformed_dag = extract_dag_info(df_transformed, "transformed")
aggregated_dag = extract_dag_info(df_aggregated, "aggregated")



EXTRACTING DAG INFORMATION


In [9]:
original_dag



{'stage_name': 'original',
 'logical_plan': '== Analyzed Logical Plan ==\nfirst_name: string, last_name: string, age: int, salary: double, department: string\nLogicalRDD [first_name#0, last_name#1, age#2, salary#3, department#4], false\n\n== Optimized Logical Plan ==\nLogicalRDD [first_name#0, last_name#1, age#2, salary#3, department#4], false\n',
 'physical_plan': '== Physical Plan ==\n*(1) Scan ExistingRDD[first_name#0,last_name#1,age#2,salary#3,department#4]\n\n',
 'full_explain': '== Parsed Logical Plan ==\nLogicalRDD [first_name#0, last_name#1, age#2, salary#3, department#4], false\n\n== Analyzed Logical Plan ==\nfirst_name: string, last_name: string, age: int, salary: double, department: string\nLogicalRDD [first_name#0, last_name#1, age#2, salary#3, department#4], false\n\n== Optimized Logical Plan ==\nLogicalRDD [first_name#0, last_name#1, age#2, salary#3, department#4], false\n\n== Physical Plan ==\n*(1) Scan ExistingRDD[first_name#0,last_name#1,age#2,salary#3,department#4]\n\

In [10]:
transformed_dag

{'stage_name': 'transformed',
 'logical_plan': '== Analyzed Logical Plan ==\nfirst_name: string, last_name: string, age: int, salary: double, department: string, full_name: string, salary_category: string, age_group: string\nProject [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, salary_category#59, CASE WHEN (age#2 < 30) THEN Young ELSE Senior END AS age_group#67]\n+- Project [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, CASE WHEN (salary#3 > cast(70000 as double)) THEN High WHEN (salary#3 > cast(50000 as double)) THEN Medium ELSE Low END AS salary_category#59]\n   +- Project [first_name#0, last_name#1, age#2, salary#3, department#4, concat(first_name#0,  , last_name#1) AS full_name#52]\n      +- LogicalRDD [first_name#0, last_name#1, age#2, salary#3, department#4], false\n\n== Optimized Logical Plan ==\nProject [first_name#0, last_name#1, age#2, salary#3, department#4, concat(first_name#0,  , last_name#1) AS full_name#52, CASE WHEN 

In [11]:
aggregated_dag

{'stage_name': 'aggregated',
 'logical_plan': '== Analyzed Logical Plan ==\ndepartment: string, salary_category: string, total_salary: double, avg_age: double, employee_count: bigint\nAggregate [department#4, salary_category#59], [department#4, salary_category#59, sum(salary#3) AS total_salary#118, avg(age#2) AS avg_age#120, count(1) AS employee_count#122L]\n+- Project [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, salary_category#59, CASE WHEN (age#2 < 30) THEN Young ELSE Senior END AS age_group#67]\n   +- Project [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, CASE WHEN (salary#3 > cast(70000 as double)) THEN High WHEN (salary#3 > cast(50000 as double)) THEN Medium ELSE Low END AS salary_category#59]\n      +- Project [first_name#0, last_name#1, age#2, salary#3, department#4, concat(first_name#0,  , last_name#1) AS full_name#52]\n         +- LogicalRDD [first_name#0, last_name#1, age#2, salary#3, department#4], false\n\n== Optimized 

**Extract transformation contexts**

In [12]:
transform_context_1 = extract_transformation_context(df_original, df_transformed, "column_derivation")
transform_context_2 = extract_transformation_context(df_transformed, df_aggregated, "aggregation")

In [13]:
transform_context_1

{'transformation_type': 'column_derivation',
 'input_columns': ['first_name', 'last_name', 'age', 'salary', 'department'],
 'output_columns': ['first_name',
  'last_name',
  'age',
  'salary',
  'department',
  'full_name',
  'salary_category',
  'age_group'],
 'new_columns': ['age_group', 'full_name', 'salary_category'],
 'dropped_columns': []}

In [14]:
transform_context_2

{'transformation_type': 'aggregation',
 'input_columns': ['first_name',
  'last_name',
  'age',
  'salary',
  'department',
  'full_name',
  'salary_category',
  'age_group'],
 'output_columns': ['department',
  'salary_category',
  'total_salary',
  'avg_age',
  'employee_count'],
 'new_columns': ['avg_age', 'total_salary', 'employee_count'],
 'dropped_columns': ['age',
  'salary',
  'age_group',
  'last_name',
  'full_name',
  'first_name']}

**Prepare data for LLM processing**

In [15]:

llm_input_data = {
    "job_id": "sample_lineage_job",
    "transformations": [
        {
            "stage": "original_to_transformed",
            "context": transform_context_1,
            "dag_info": transformed_dag
        },
        {
            "stage": "transformed_to_aggregated",
            "context": transform_context_2,
            "dag_info": aggregated_dag
        }
    ]
}

**DAG information for LLM**

In [16]:

print("\nSample DAG Information for LLM:")
print(json.dumps(llm_input_data, indent=2)[:1000] + "...")


print("\nLogical Plan for 'transformed' stage:")
print(transformed_dag["logical_plan"][:500] + "...")

print("\nPhysical Plan for 'aggregated' stage:")
print(aggregated_dag["physical_plan"][:500] + "...")


print("\n" + "="*50)
print("DAG EXTRACTION COMPLETE")
print("="*50)


Sample DAG Information for LLM:
{
  "job_id": "sample_lineage_job",
  "transformations": [
    {
      "stage": "original_to_transformed",
      "context": {
        "transformation_type": "column_derivation",
        "input_columns": [
          "first_name",
          "last_name",
          "age",
          "salary",
          "department"
        ],
        "output_columns": [
          "first_name",
          "last_name",
          "age",
          "salary",
          "department",
          "full_name",
          "salary_category",
          "age_group"
        ],
        "new_columns": [
          "age_group",
          "full_name",
          "salary_category"
        ],
        "dropped_columns": []
      },
      "dag_info": {
        "stage_name": "transformed",
        "logical_plan": "== Analyzed Logical Plan ==\nfirst_name: string, last_name: string, age: int, salary: double, department: string, full_name: string, salary_category: string, age_group: string\nProject [first_

**Function to prepare LLM prompt**

In [17]:
def prepare_llm_prompt(transformation_data):
    """Prepare structured prompt for LLM to extract column lineage"""

    prompt = f"""
    Analyze this Spark transformation and extract column lineage:

    Transformation Type: {transformation_data['context']['transformation_type']}
    Input Columns: {transformation_data['context']['input_columns']}
    Output Columns: {transformation_data['context']['output_columns']}
    New Columns: {transformation_data['context']['new_columns']}

    Logical Plan:
    {transformation_data['dag_info']['logical_plan'][:800]}

    Extract column lineage in this format:
    {{
        "lineage": [
            {{"source_columns": ["col1", "col2"], "target_column": "derived_col", "operation": "concat"}},
            {{"source_columns": ["col3"], "target_column": "derived_col2", "operation": "case_when"}}
        ]
    }}
    """

    return prompt

In [18]:
sample_prompt = prepare_llm_prompt(llm_input_data["transformations"][0])

In [19]:
sample_prompt

'\n    Analyze this Spark transformation and extract column lineage:\n\n    Transformation Type: column_derivation\n    Input Columns: [\'first_name\', \'last_name\', \'age\', \'salary\', \'department\']\n    Output Columns: [\'first_name\', \'last_name\', \'age\', \'salary\', \'department\', \'full_name\', \'salary_category\', \'age_group\']\n    New Columns: [\'age_group\', \'full_name\', \'salary_category\']\n\n    Logical Plan:\n    == Analyzed Logical Plan ==\nfirst_name: string, last_name: string, age: int, salary: double, department: string, full_name: string, salary_category: string, age_group: string\nProject [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, salary_category#59, CASE WHEN (age#2 < 30) THEN Young ELSE Senior END AS age_group#67]\n+- Project [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, CASE WHEN (salary#3 > cast(70000 as double)) THEN High WHEN (salary#3 > cast(50000 as double)) THEN Medium ELSE Low END AS salary

**LLMA model**

In [20]:
!pip install transformers==4.36.0 torch accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re



/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [22]:
from google.colab import userdata
from huggingface_hub import login

token = userdata.get('HF_TOKEN')
login(token=token, add_to_git_credential=False)

**Model loading**

In [23]:
print("Loading CodeLlama model...")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/CodeLlama-7b-Instruct-hf")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/CodeLlama-7b-Instruct-hf",
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_8bit=True  # For Colab memory optimization
)

Loading CodeLlama model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [24]:
def process_prompt_with_codellama(prompt):
    """Process prompt through CodeLlama"""
    formatted_prompt = f"<s>[INST] {prompt.strip()} [/INST]"

    inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=2048)

    # Move inputs to GPU
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_new_tokens=512,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_start = response.find("[/INST]") + 7
    return response[response_start:].strip()

In [25]:
test_response = process_prompt_with_codellama(sample_prompt)
print("LLM Response:", test_response[:200])

LLM Response: The column lineage for this Spark transformation can be extracted as follows:

{
    "lineage": [
        {"source_columns": ["first_name", "last_name"], "target_column": "full_name", "operation": "co


In [26]:
sample_prompt

'\n    Analyze this Spark transformation and extract column lineage:\n\n    Transformation Type: column_derivation\n    Input Columns: [\'first_name\', \'last_name\', \'age\', \'salary\', \'department\']\n    Output Columns: [\'first_name\', \'last_name\', \'age\', \'salary\', \'department\', \'full_name\', \'salary_category\', \'age_group\']\n    New Columns: [\'age_group\', \'full_name\', \'salary_category\']\n\n    Logical Plan:\n    == Analyzed Logical Plan ==\nfirst_name: string, last_name: string, age: int, salary: double, department: string, full_name: string, salary_category: string, age_group: string\nProject [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, salary_category#59, CASE WHEN (age#2 < 30) THEN Young ELSE Senior END AS age_group#67]\n+- Project [first_name#0, last_name#1, age#2, salary#3, department#4, full_name#52, CASE WHEN (salary#3 > cast(70000 as double)) THEN High WHEN (salary#3 > cast(50000 as double)) THEN Medium ELSE Low END AS salary

In [27]:
test_response

'The column lineage for this Spark transformation can be extracted as follows:\n\n{\n    "lineage": [\n        {"source_columns": ["first_name", "last_name"], "target_column": "full_name", "operation": "concat"},\n        {"source_columns": ["salary"], "target_column": "salary_category", "operation": "case_when"},\n        {"source_columns": ["age"], "target_column": "age_group", "operation": "case_when"}\n    ]\n}\n\nHere, we have three column derivations:\n\n1. The "full_name" column is derived from the "first_name" and "last_name" columns using the "concat" operation.\n2. The "salary_category" column is derived from the "salary" column using the "case_when" operation.\n3. The "age_group" column is derived from the "age" column using the "case_when" operation.\n\nNote that the "case_when" operation is used for both the "salary_category" and "age_group" columns, but the source columns and the target column names are different for each operation.'